In [1]:
%%time
EXCHANGE = 'huobi'
PROJECT_ID = 'algebraic-ward-278316'

# get top markets
def top_markets(exchange=EXCHANGE):
    # client
    from coinpaprika import client as Coinpaprika
    client = Coinpaprika.Client()
    coins  = client.coins()
    markets = client.exchange_markets(exchange, quotes="USD")
    return [m['pair'] for c in coins for m in markets if (m['base_currency_id'] == c['id'])]

tm = top_markets()

# pull ohlcv from ccxt
import ccxt
import numpy as np
import pandas as pd
dfs=[]
for m in tm[:5]:
    try:
        df = pd.DataFrame(np.array(ccxt.huobipro().fetch_ohlcv(m, timeframe='1m', limit=60)),\
                     columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])\
                     .astype({'timestamp':'datetime64[ms]'})
    except:
        continue
    df.insert(0, 'symbol', m, True)
    dfs.append(df) 
ohlcv = pd.concat(dfs, axis=0)

CPU times: user 1.15 s, sys: 260 ms, total: 1.41 s
Wall time: 11.8 s


In [ ]:
import pandas_gbq
pandas_gbq.to_gbq(ohlcv, f'ohlcv.{EXCHANGE}', project_id=PROJECT_ID, if_exists='append')